In [101]:
import requests
from lxml import html as lhtml
from tqdm.notebook import tqdm
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd

In [60]:
with open('authors.txt') as authors:
        authors_ids = [int(line.rstrip()) for line in authors]

In [88]:
def get_page(url, attempts = 3):
    for get in range(attempts):
        page = requests.get(url)
        if not page.ok:
            print(page.status_code)
        else:
            return page
        time.sleep(1)
    print("Error in getting {} page".format(url))

In [89]:
def get_authors_books1(author_id):
    book_urls=[]
    path_for_books = '//div[@class="rd-page-listing__two-col_right"]/div[@class="rd-page-listing__products"]/div[contains(@class,rd-listing-product-item-data-wrap)]/div/a/attribute::href'
    path_for_num = '//div[@class="rd-page-listing__three-col"]/div/div/span[@class="rd-listing-count__total"]/text()'
    page = get_page('https://www.respublica.ru/authors/{}?page=1'.format(author_id))
    tree = lhtml.fromstring(page.text)
    urls = tree.xpath(path_for_books)
    num_of_items = tree.xpath(path_for_num)
    for url in urls:
        book_urls.append('https://www.respublica.ru' + url)
    num_of_items = int(num_of_items[0].rstrip())
    for page_num in range(2,ceil(num_of_items/22)+1):
        page = get_page('https://www.respublica.ru/authors/{}?page={}'.format(author_id,page_num))
        tree = lhtml.fromstring(page.text)
        urls = tree.xpath(path_for_books)
        for url in urls:
            book_urls.append('https://www.respublica.ru' + url)
    return book_urls

In [90]:
def get_authors_books(author_id):
    book_urls=[]
    path_for_books = '//div[@class="rd-page-listing__two-col_right"]/div[@class="rd-page-listing__products"]/div[contains(@class,rd-listing-product-item-data-wrap)]/div/a/attribute::href'
    path_for_num = '//div[@class="rd-page-listing__three-col"]/div/div/span[@class="rd-listing-count__total"]/text()'
    page = get_page('https://www.respublica.ru/authors/{}?page=1'.format(author_id))
    tree = lhtml.fromstring(page.text)
    urls = tree.xpath(path_for_books)
    num_of_items = tree.xpath(path_for_num)
    for url in urls:
        book_urls.append('https://www.respublica.ru' + url)
    num_of_items = int(num_of_items[0].rstrip())
    page_num = 2
    while(True):
        page = get_page('https://www.respublica.ru/authors/{}?page={}'.format(author_id,page_num))
        tree = lhtml.fromstring(page.text)
        urls = tree.xpath(path_for_books)
        if len(urls) == 0:
            return book_urls
        for url in urls:
            book_urls.append('https://www.respublica.ru' + url)
        page_num += 1
    return book_urls

In [91]:
urls = []
with Pool(10) as pool:
    with tqdm(total=len(authors_ids)) as pbar:
        for i, url in enumerate(pool.imap_unordered(get_authors_books,authors_ids)):
            urls.extend(url)
            pbar.update()
pool.join()

In [92]:
len(set(urls))

2455

In [93]:
def get_card(url):
    card = dict()
    page = get_page(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    # Разделим страницу на три части
    left_attrs = soup.find('div',class_='rd-page-product__col-left')
    right_attrs = soup.find('div',class_='rd-page-product__col-right')
    bottom = soup.find('div',class_="rd-page-product__row rd-page-product__desc")
    card['ID'] = left_attrs.find('span',itemprop = "sku").text
    card['Цена'] = int(right_attrs.find('span',class_='num').text.replace(' ',''))
    card['Название'] = left_attrs.find('h1',itemprop = "name").text
    card['URL'] = url
    preview = left_attrs.find('a',class_ = "download-pdf")
    if preview is not None:
        card['Превью'] = 'https://www.respublica.ru' + preview.attrs['href']
    card['Изображение'] = 'https://www.respublica.ru' + \
        left_attrs.find('img',class_="rd-page-product__img").attrs['src']
    card['Автор'] = left_attrs.find('a',itemprop = "brand").text
    a = soup.find_all('span',class_="rd-page-breadcrumbs-item")
    a = [e.text for e in soup.find_all('span',class_="rd-page-breadcrumbs-item")]
    card['Категория'] = '; '.join([item[1:] for item in a])
    rating_info = right_attrs.find('span', itemprop = "aggregateRating")
    if rating_info is not None:
        card['Количество отзывов'] = int(rating_info.find('meta',itemprop = "reviewCount").attrs['content'])
        card['Количество оценок'] = int(rating_info.find('meta',itemprop = "ratingCount").attrs['content'])
        card['Оценка'] = float(rating_info.find('meta',itemprop = "ratingValue").attrs['content'])
    else:
        card['Количество отзывов'] = 0
        card['Количество оценок'] = 0
        card['Оценка'] = 0
    old_price = right_attrs.find('div',class_="rd-page-product__price-old")
    if old_price is not None:
        card['Цена(старая)'] = int(old_price.find_next('span',class_="prev").text[:-5].replace(' ',''))
    button_text = right_attrs.find('span',class_="rd-page-product__buy-text").text
    if button_text == "Купить" or button_text == "Купить со скидкой":
        card['Наличие'] = True
    else:
        card['Наличие'] = False
    description = bottom.find('div',class_="rd-page-product__col-left")
    text_tag = description.find('p')
    if text_tag is not None:
        card['Описание'] = text_tag.text
    else:
        card['Описание'] = text_tag = description.find('div', itemprop = "description").text    
    table = bottom.find('div',class_="rd-page-product__desc-params")
    rows = table.find_all('p',class_="rd-page-product__desc-param")
    for row in rows:
        name = row.find('span',itemprop="name").text
        non_url_value = row.find('span',itemprop="value")
        if non_url_value is None:
            card[name] = row.find('a',itemprop="value").text
        else:
            card[name] = non_url_value.text
    return card

In [94]:
cards = []
with Pool(10) as pool:
    with tqdm(total=len(urls)) as pbar:
        for i, card in enumerate(pool.imap_unordered(get_card,urls)):
            cards.append(card)
            pbar.update()
pool.join()

In [95]:
cards

[{'ID': '456735',
  'Цена': 120,
  'Название': 'Капитанская дочка',
  'URL': 'https://www.respublica.ru/knigi/hudozhestvennaya-literatura/klassicheskaya-literatura/456735-kapitanskaya-dochka',
  'Превью': 'https://www.respublica.ru/items/332328/download_preview_pdf',
  'Изображение': 'https://www.respublica.ru/uploads/00/00/00/4i/ew/large_755673e6de09e2c1.jpg',
  'Автор': 'Пушкин Александр Сергеевич',
  'Категория': 'Книги; Художественная литература; Классическая литература',
  'Количество отзывов': 0,
  'Количество оценок': 0,
  'Оценка': 0,
  'Наличие': True,
  'Описание': '"Капитанская дочка" А. Пушкина - одно из важнейших произведений русской классики, провозгласившей главным жизненным принципом поговорку "Береги честь с молоду". Н. Гоголь восторженно отозвался о "Капитанской дочке":"Решительно лучшее русское произведенье в повествовательном роде. Сравнительно с "Капитанской дочкой" все наши романы и повести кажутся приторной размазней. <...> В первый раз выступили истинно русские 

In [99]:
df = pd.DataFrame(cards)
df.sort_values(by=['ID'], inplace=True)

In [100]:
df

,ID,Цена,Название,URL,Превью,Изображение,Автор,Категория,Количество отзывов,Количество оценок,...,Эпоха,Упаковка,Материал,Рисунок,"Размер, см",Вид бумаги,Пол,Назначение,Тематика,Иллюстрации
2065,000033,100,Много шума из ничего,https://www.respublica.ru/knigi/hudozhestvenna...,NaN,https://www.respublica.ru/uploads/00/00/00/cq/...,Уильям Шекспир,Книги; Художественная литература; Поэзия,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2064,000056,100,Двенадцатая ночь,https://www.respublica.ru/knigi/hudozhestvenna...,NaN,https://www.respublica.ru/uploads/01/00/00/24/...,Уильям Шекспир,Книги; Художественная литература; Поэзия,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1848,021678,610,N or M?,https://www.respublica.ru/knigi/hudozhestvenna...,NaN,https://www.respublica.ru/uploads/00/00/00/68/...,Agatha Christie,Книги; Художественная литература; Книги на ино...,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1847,021679,610,The Secret Adversary,https://www.respublica.ru/knigi/hudozhestvenna...,NaN,https://www.respublica.ru/uploads/00/00/00/68/...,Agatha Christie,Книги; Художественная литература; Книги на ино...,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1874,021680,510,By the Pricking of My Thumbs,https://www.respublica.ru/knigi/hudozhestvenna...,NaN,https://www.respublica.ru/uploads/01/00/00/68/...,Agatha Christie,Книги; Художественная литература; Книги на ино...,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2420,532529,680,Приключения Эмиля из Лённеберги,https://www.respublica.ru/knigi/detskie-knigi/...,NaN,https://www.respublica.ru/uploads/01/00/00/dp/...,Астрид Линдгрен,Книги; Детские книги; Художественная литература,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1777,532971,860,Загадочный гость,https://www.respublica.ru/knigi/detskie-knigi/...,NaN,https://www.respublica.ru/uploads/00/00/00/dr/...,Льюис Кэрролл,Книги; Детские книги; Художественная литература,0,0,...,NaN,NaN,NaN,NaN,NaN,Мелованная,NaN,NaN,NaN,Цветные
2314,532974,1310,Рассказы,https://www.respublica.ru/knigi/hudozhestvenna...,NaN,https://www.respublica.ru/uploads/00/00/00/dr/...,Антон Павлович Чехов,Книги; Художественная литература; Классическая...,0,0,...,NaN,NaN,NaN,NaN,NaN,Мелованная,NaN,NaN,NaN,Цветные
1983,8121280,430,Завтрашний день кошки,https://www.respublica.ru/knigi/hudozhestvenna...,NaN,https://www.respublica.ru/uploads/01/00/00/6h/...,Бернар Вербер,Книги; Художественная литература; Современная ...,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
with open('hw_3.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)